In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [12]:
count=0
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["release_year"])
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_l4,Approved,KEGG,Ingredient,date,release_year,成分,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,3.261297e+08
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,2.377048e+08
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,6.583945e+08
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,6.363609e+08
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,1147,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1,1147.0,3.368936e+08


後発区分を廃す

1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [13]:
data["elapsed"]=data["year"]-data["release_year"]
#　不要なところを除外
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","id"])[sumcolumns].sum().reset_index()
# save temporarily
colnames=dummies.columns.to_list()
if count:
    # count ver.
    aggregated_df= pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()
else:
    # dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id']).first(), 
        dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
# sumをとったところを追加する
aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df
aggregated_df.head()

,year,id,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,...,elapsed_22,elapsed_23,elapsed_24,elapsed_25,elapsed_26,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31
0,2014,1147.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,1.652349e+08,...,0,0,0,0,0,0,0,0,0,0
1,2014,1149.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,0.0,1.632358e+10,...,0,0,0,0,0,0,0,0,0,0
2,2014,1219.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,0.0,4.840017e+07,...,0,0,0,0,0,0,0,0,0,0
3,2014,1231.0,123.0,自律神経剤,620002526.0,チアトンカプセル１０ｍｇ,1231013M2179,15.2,0.0,1.596449e+08,...,0,0,0,0,0,0,0,0,0,0
4,2014,1319.0,131.0,眼科用剤,620207901.0,アダプチノール錠５ｍｇ,1319004F1035,45.9,0.0,8.960133e+09,...,0,0,0,0,0,0,0,0,0,0


connect generic usage

In [14]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len4_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric

,id,year,generic_share_q,generic_share_r
0,1147.0,2014,0.076961,0.059379
33,1149.0,2014,0.244198,0.053661
80,1219.0,2014,0.062114,0.053451
96,1231.0,2014,0.129137,0.064243
147,1319.0,2014,0.136869,0.080471
...,...,...,...,...
25142,4413.0,2021,0.342768,0.263456
25182,4490.0,2021,0.608051,0.363450
25386,6250.0,2021,0.502607,0.052158
25628,6290.0,2021,0.284349,0.073842


In [15]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["id","year"])
merged_df.head()

,year,id,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,...,elapsed_26,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31,Year,Percentage,generic_share_q,generic_share_r
0,2014,1147.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,1.652349e+08,...,0,0,0,0,0,0,2014,51.55,0.076961,0.059379
1,2014,1149.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,0.0,1.632358e+10,...,0,0,0,0,0,0,2014,51.55,0.244198,0.053661
2,2014,1219.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,0.0,4.840017e+07,...,0,0,0,0,0,0,2014,51.55,0.062114,0.053451
3,2014,1231.0,123.0,自律神経剤,620002526.0,チアトンカプセル１０ｍｇ,1231013M2179,15.2,0.0,1.596449e+08,...,0,0,0,0,0,0,2014,51.55,0.129137,0.064243
4,2014,1319.0,131.0,眼科用剤,620207901.0,アダプチノール錠５ｍｇ,1319004F1035,45.9,0.0,8.960133e+09,...,0,0,0,0,0,0,2014,51.55,0.136869,0.080471


In [16]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies.head()

,year,id,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,...,id_2659.0,id_3327.0,id_3399.0,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0
0,2014,1147.0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,1.652349e+08,...,0,0,0,0,0,0,0,0,0,0
1,2014,1149.0,114.0,解熱鎮痛消炎剤,620098801.0,ロキソニン錠６０ｍｇ,1149019F1560,17.5,0.0,1.632358e+10,...,0,0,0,0,0,0,0,0,0,0
2,2014,1219.0,121.0,局所麻酔剤,620005991.0,ストロカイン錠５ｍｇ,1219002F1065,5.7,0.0,4.840017e+07,...,0,0,0,0,0,0,0,0,0,0
3,2014,1231.0,123.0,自律神経剤,620002526.0,チアトンカプセル１０ｍｇ,1231013M2179,15.2,0.0,1.596449e+08,...,0,0,0,0,0,0,0,0,0,0
4,2014,1319.0,131.0,眼科用剤,620207901.0,アダプチノール錠５ｍｇ,1319004F1035,45.9,0.0,8.960133e+09,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
if count:
    df_with_dummies.to_csv("../merged/len4_ndb_sum_agg_all_DN.csv",encoding="shift-jis")
else:
    df_with_dummies.to_csv("../merged/len4_ndb_agg_all_DropNever.csv",encoding="shift-jis")